In [4]:
import pandas as pd
import glob
import os
import re
import gc # Garbage Collector untuk hemat RAM
import warnings

warnings.filterwarnings('ignore')

# --- FUNGSI NORMALISASI (Wajib Sama dengan Sebelumnya) ---
def normalize_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    name = re.sub(r'[/\-._]', ' ', name)
    name = name.replace('FORKLIF ', 'FORKLIFT ')
    if name.endswith('FORKLIF'): name = name + 'T'
    name = " ".join(name.split())
    return name

# ==============================================================================
# TAHAP 1: MEMBANGUN KAMUS "JENIS ALAT"
# ==============================================================================
print("🚀 [1/3] MEMBANGUN KAMUS DATA (UNIT -> JENIS ALAT)...")

base_folder = "Data Setiap Cabang"
if not os.path.exists(base_folder):
    branch_files = glob.glob("*Rpt HMU*.csv") + glob.glob("*Rpt HMU*.xlsx")
else:
    search_pattern_csv = os.path.join(base_folder, "**", "*Rpt HMU*.csv")
    search_pattern_xlsx = os.path.join(base_folder, "**", "*Rpt HMU*.xlsx")
    branch_files = glob.glob(search_pattern_csv, recursive=True) + glob.glob(search_pattern_xlsx, recursive=True)

# Dictionary ringan untuk menyimpan pemetaan
unit_details_map = {}

for i, f in enumerate(branch_files):
    try:
        # Baca file
        if f.endswith('.csv'): 
            # Hanya baca kolom yang diperlukan agar hemat RAM
            df = pd.read_csv(f, usecols=lambda x: x.upper().strip() in ['EQUIP NAME', 'JENIS ALAT', 'PORT', 'LOKASI'])
        else: 
            df = pd.read_excel(f, engine='openpyxl') # Excel susah filter kolom di awal, baca dulu
        
        df.columns = [str(c).strip().upper() for c in df.columns]
        
        if 'EQUIP NAME' in df.columns and 'JENIS ALAT' in df.columns:
            # Ambil data unik saja (drop duplicates) agar proses cepat
            cols_to_keep = ['EQUIP NAME', 'JENIS ALAT']
            if 'PORT' in df.columns: cols_to_keep.append('PORT')
            if 'LOKASI' in df.columns: cols_to_keep.append('LOKASI')
            
            df_unique = df[cols_to_keep].drop_duplicates()
            
            for index, row in df_unique.iterrows():
                raw_name = row['EQUIP NAME']
                jenis_alat = str(row['JENIS ALAT']).upper().strip()
                clean_name = normalize_unit_name(raw_name)
                
                # Simpan ke dictionary jika belum ada
                if clean_name and clean_name not in unit_details_map:
                    info = {'Jenis_Alat': jenis_alat}
                    if 'PORT' in df.columns: info['Port_Cabang'] = str(row['PORT']).upper().strip()
                    if 'LOKASI' in df.columns: info['Lokasi_Kerja'] = str(row['LOKASI']).upper().strip()
                    unit_details_map[clean_name] = info
        
        # BERSIHKAN MEMORI
        del df
        if 'df_unique' in locals(): del df_unique
        gc.collect()
        
    except Exception as e:
        print(f"   ⚠️ Skip file error: {os.path.basename(f)}")

print(f"✅ Kamus Data Siap. Teridentifikasi {len(unit_details_map)} unit unik dari Cabang.")

🚀 [1/3] MEMBANGUN KAMUS DATA (UNIT -> JENIS ALAT)...
✅ Kamus Data Siap. Teridentifikasi 245 unit unik dari Cabang.


In [5]:
# ==============================================================================
# TAHAP 2: MENERAPKAN KE DATA BBM (HEMAT RAM)
# ==============================================================================
print("🚀 [2/3] MENCOCOKKAN DENGAN DATA BBM AAB...")

bbm_files = glob.glob("BBM AAB.xlsx") + glob.glob("BBM AAB.xlsx - *.csv")
final_unit_list = []
seen_units_bbm = set()

import numpy as np

for f in bbm_files:
    try:
        # Load File
        if f.endswith('.xlsx'):
            xls = pd.ExcelFile(f)
            sheet_names = xls.sheet_names
        else:
            sheet_names = ['Sheet1'] # Dummy
            
        for sheet in sheet_names:
            # Baca Header Saja (Baris 0-5) untuk ambil Nama Unit
            # Kita tidak perlu data angka HM/Liter disini, jadi baca sedikit saja cukup
            if f.endswith('.xlsx'): 
                df_raw = pd.read_excel(xls, sheet_name=sheet, header=None, nrows=5)
            else: 
                df_raw = pd.read_csv(f, header=None, nrows=5)
            
            # Ambil Baris Nama Unit (Baris 0)
            row0 = df_raw.iloc[0].replace(r'^\s*$', np.nan, regex=True)
            row_units = row0.fillna(method='ffill').astype(str)
            
            for val in row_units:
                raw_name = str(val).strip().upper()
                
                # Filter Sampah
                invalid = ['NAN', 'UNNAMED', 'EQUIP NAME', 'TANGGAL', 'GROUP KPI', 'NAT', 'TOTAL', 'LITER', 'HM', 'PHOTOS', 'KET']
                is_invalid = any(x in raw_name for x in invalid)
                
                if not is_invalid and len(raw_name) > 2:
                    clean_name = normalize_unit_name(raw_name)
                    
                    if clean_name not in seen_units_bbm:
                        seen_units_bbm.add(clean_name)
                        
                        # --- AMBIL INFO DARI KAMUS (CELL 1) ---
                        mapped_info = unit_details_map.get(clean_name, {})
                        
                        final_unit_list.append({
                            'Unit_Name_BBM': clean_name,       # Nama Standar (Key)
                            'Original_Name_BBM': raw_name,     # Nama Asli di BBM
                            'Jenis_Alat': mapped_info.get('Jenis_Alat', 'UNKNOWN'),
                            'Port_Cabang': mapped_info.get('Port_Cabang', '-'),
                            'Lokasi_Kerja': mapped_info.get('Lokasi_Kerja', '-')
                        })
            
            # Bersihkan Memori per Sheet
            del df_raw
            del row_units
            gc.collect()

    except Exception as e:
        print(f"   ⚠️ Error baca {f}: {e}")

print(f"✅ Mapping Selesai. Total {len(final_unit_list)} unit BBM berhasil dicek.")

🚀 [2/3] MENCOCOKKAN DENGAN DATA BBM AAB...
✅ Mapping Selesai. Total 330 unit BBM berhasil dicek.


In [6]:
# ==============================================================================
# TAHAP 3: EXPORT MASTER DATA
# ==============================================================================
print("🚀 [3/3] MENYIMPAN HASIL KE EXCEL...")

if final_unit_list:
    df_master = pd.DataFrame(final_unit_list)
    
    # Sortir:
    # 1. Jenis Alat (UNKNOWN di paling bawah agar mudah dicek)
    # 2. Nama Unit
    df_master.sort_values(['Jenis_Alat', 'Unit_Name_BBM'], inplace=True)
    
    # Simpan ke File Terpisah (Sesuai Request)
    output_file = 'Master_Mapping_Jenis_Alat.xlsx'
    df_master.to_excel(output_file, index=False)
    
    print(f"\n💾  SUKSES! File tersimpan: '{output_file}'")
    print(f"    - Total Populasi Unit BBM: {len(df_master)}")
    
    # Hitung Statistik
    unknown_count = len(df_master[df_master['Jenis_Alat'] == 'UNKNOWN'])
    known_count = len(df_master) - unknown_count
    
    print(f"    - Teridentifikasi Jenisnya: {known_count} unit")
    print(f"    - Tidak Dikenali (UNKNOWN): {unknown_count} unit")
    print("      (Unit UNKNOWN adalah unit yang ada di BBM tapi tidak ditemukan di File Cabang manapun)")
    
else:
    print("❌ Tidak ada unit yang berhasil diproses.")

🚀 [3/3] MENYIMPAN HASIL KE EXCEL...

💾  SUKSES! File tersimpan: 'Master_Mapping_Jenis_Alat.xlsx'
    - Total Populasi Unit BBM: 330
    - Teridentifikasi Jenisnya: 242 unit
    - Tidak Dikenali (UNKNOWN): 88 unit
      (Unit UNKNOWN adalah unit yang ada di BBM tapi tidak ditemukan di File Cabang manapun)


## ANALISA BENCHMARK PER JENIS ALAT BERAT (FILE CABANG)

In [3]:
import pandas as pd
import numpy as np
import os
import re
import gc
import warnings

warnings.filterwarnings('ignore')

# ==============================================================================
# BAGIAN 1: SETUP & LOAD MASTER MAPPING
# ==============================================================================
print("🚀 [1/3] MEMBACA MASTER DATA JENIS ALAT...")

def normalize_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    name = re.sub(r'[/\-._]', ' ', name)
    name = name.replace('FORKLIF ', 'FORKLIFT ')
    if name.endswith('FORKLIF'): name = name + 'T'
    name = " ".join(name.split())
    return name

mapping_file = 'Master_Mapping_Jenis_Alat.xlsx'
unit_type_map = {}

# Cek keberadaan file mapping (Excel atau CSV)
if os.path.exists(mapping_file):
    df_map = pd.read_excel(mapping_file)
elif os.path.exists(mapping_file.replace('.xlsx', '.csv')):
    df_map = pd.read_csv(mapping_file.replace('.xlsx', '.csv'))
else:
    df_map = pd.DataFrame()

if not df_map.empty:
    for _, row in df_map.iterrows():
        # Ambil kolom Unit dan Jenis (sesuaikan index kolom jika perlu)
        unit = str(row.get('Unit_Name_BBM', row.iloc[0])) 
        jenis = str(row.get('Jenis_Alat', row.iloc[2]))
        
        if jenis.upper() != 'UNKNOWN':
            unit_type_map[normalize_unit_name(unit)] = jenis
            
    print(f"✅ Master Data Terbaca. Total Unit Terdaftar: {len(unit_type_map)}")
else:
    print(f"❌ File Mapping '{mapping_file}' tidak ditemukan! Kategori alat mungkin kosong.")

🚀 [1/3] MEMBACA MASTER DATA JENIS ALAT...
✅ Master Data Terbaca. Total Unit Terdaftar: 242


In [8]:
# ==============================================================================
# BAGIAN 2: BACA EXCEL (XLSX) & HITUNG DELTA HARIAN (REVISI LOGIC)
# ==============================================================================
print("🚀 [2/3] MEMPROSES FILE EXCEL 'BBM AAB.xlsx'...")

filename_excel = 'BBM AAB.xlsx'
target_sheets = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 'JUL', 'AGT', 'SEP', 'OKT', 'NOV']
raw_data_list = []

if os.path.exists(filename_excel):
    try:
        # Buka File Excel Sekali
        xls = pd.ExcelFile(filename_excel)
        print(f"   -> File ditemukan. Sheet tersedia: {len(xls.sheet_names)}")
        
        for sheet in target_sheets:
            if sheet in xls.sheet_names:
                print(f"   -> Membaca Sheet: {sheet}")
                
                # Baca Sheet tanpa header (Baris 0=Unit, Baris 2=Metric)
                df = pd.read_excel(xls, sheet_name=sheet, header=None)
                
                # --- PARSING STRUKTUR (SAMA SEPERTI NOTEBOOK) ---
                unit_names = df.iloc[0].ffill() # Baris 0: Nama Unit
                headers = df.iloc[2]            # Baris 2: Metric (HM/LITER)
                dates = df.iloc[3:, 0]          # Kolom 0: Tanggal
                
                # Loop Kolom Data (Mulai kolom 1)
                for col in range(1, df.shape[1]):
                    header_str = str(headers[col]).strip().upper()
                    
                    # Filter Kolom: HM, LITER, KELUAR
                    if header_str in ['HM', 'LITER', 'KELUAR', 'PEMAKAIAN']:
                        metric_type = 'HM' if header_str == 'HM' else 'LITER'
                        
                        unit_raw = str(unit_names[col])
                        unit_clean = normalize_unit_name(unit_raw)
                        
                        # Hanya proses jika unit ada di Master Mapping
                        if unit_clean in unit_type_map:
                            vals = pd.to_numeric(df.iloc[3:, col], errors='coerce')
                            
                            temp_df = pd.DataFrame({
                                'Date': dates,
                                'Month': sheet, # Info bulan
                                'Unit_Name': unit_clean,
                                'Metric': metric_type,
                                'Value': vals
                            })
                            
                            temp_df.dropna(subset=['Value', 'Date'], inplace=True)
                            if not temp_df.empty:
                                raw_data_list.append(temp_df)
                
                del df; gc.collect()
            else:
                print(f"      ⚠️ Sheet '{sheet}' tidak ditemukan.")
                
    except Exception as e:
        print(f"   ❌ Error membaca Excel: {e}")
else:
    print(f"   ❌ File '{filename_excel}' TIDAK DITEMUKAN di folder ini.")

# --- PROSES PERHITUNGAN (LOGIKA REVISI: TANPA FILTER 24 JAM) ---
if raw_data_list:
    print("   -> Menggabungkan Data & Pivot...")
    df_all = pd.concat(raw_data_list, ignore_index=True)
    
    # Konversi Tanggal
    df_all['Date'] = pd.to_datetime(df_all['Date'], dayfirst=True, errors='coerce')
    df_all.dropna(subset=['Date'], inplace=True)
    
    # 1. PIVOT (Baris=Unit+Tgl, Kolom=HM,LITER)
    df_pivot = df_all.pivot_table(
        index=['Unit_Name', 'Date'],
        columns='Metric',
        values='Value',
        aggfunc='sum'
    ).reset_index()
    
    if 'HM' not in df_pivot.columns: df_pivot['HM'] = 0
    if 'LITER' not in df_pivot.columns: df_pivot['LITER'] = 0
    
    # Sortir Kronologis (Wajib buat Delta)
    df_pivot.sort_values(by=['Unit_Name', 'Date'], inplace=True)
    
    print("   -> Menghitung Delta HM Harian...")
    # 2. HITUNG DELTA HM (Hari Ini - Kemarin)
    df_pivot['Delta_HM'] = df_pivot.groupby('Unit_Name')['HM'].diff()
    
    # Cleaning Delta:
    df_pivot.loc[df_pivot['Delta_HM'] < 0, 'Delta_HM'] = 0 # Reset/Error jadi 0
    df_pivot['Delta_HM'] = df_pivot['Delta_HM'].fillna(0)  # Hari pertama jadi 0
    
    # --- PERBAIKAN DI SINI ---
    # Filter > 24 jam DIHAPUS agar akumulasi jam kerja (seperti LANDAK & BOSS 3) tetap terhitung.
    # df_pivot.loc[df_pivot['Delta_HM'] > 24, 'Delta_HM'] = 0 
    
    # 3. AGREGASI TOTAL (JAN-NOV)
    final_stats = df_pivot.groupby('Unit_Name').agg({
        'LITER': 'sum',
        'Delta_HM': 'sum'
    }).reset_index()
    
    final_stats.rename(columns={'LITER': 'Total_Liter', 'Delta_HM': 'Total_HM_Work'}, inplace=True)
    
    # Filter Output
    unit_perf_data = final_stats[
        (final_stats['Total_Liter'] > 0) | (final_stats['Total_HM_Work'] > 0)
    ].to_dict('records')
    
    for rec in unit_perf_data:
        rec['Jenis_Alat'] = unit_type_map.get(rec['Unit_Name'], 'UNKNOWN')
        
    print(f"✅ Selesai. Data HM & BBM dihitung dari XLSX Sheet JAN-NOV.")
    print(f"   Total Unit Terproses: {len(unit_perf_data)}")
    
else:
    print("❌ Tidak ada data yang berhasil diekstrak.")
    unit_perf_data = []

🚀 [2/3] MEMPROSES FILE EXCEL 'BBM AAB.xlsx'...
   -> File ditemukan. Sheet tersedia: 11
   -> Membaca Sheet: JAN
   -> Membaca Sheet: FEB
   -> Membaca Sheet: MAR
   -> Membaca Sheet: APR
   -> Membaca Sheet: MEI
   -> Membaca Sheet: JUN
   -> Membaca Sheet: JUL
   -> Membaca Sheet: AGT
   -> Membaca Sheet: SEP
   -> Membaca Sheet: OKT
   -> Membaca Sheet: NOV
   -> Menggabungkan Data & Pivot...
   -> Menghitung Delta HM Harian...
✅ Selesai. Data HM & BBM dihitung dari XLSX Sheet JAN-NOV.
   Total Unit Terproses: 240


In [10]:
# ==============================================================================
# BAGIAN 3: ANALISA BENCHMARK & EXPORT
# ==============================================================================
print("🚀 [3/3] MEMBUAT LAPORAN EXCEL...")

if unit_perf_data:
    df_raw = pd.DataFrame(unit_perf_data)
    
    # 1. Hitung Ratio
    df_raw['Fuel_Ratio'] = df_raw.apply(
        lambda x: x['Total_Liter'] / x['Total_HM_Work'] if x['Total_HM_Work'] > 0 else 0, axis=1
    )
    
    # 2. Benchmark (Median)
    df_valid = df_raw[(df_raw['Total_HM_Work'] > 0) & (df_raw['Total_Liter'] > 0)].copy()
    median_stats = df_valid.groupby('Jenis_Alat')['Fuel_Ratio'].median().reset_index(name='Group_Benchmark_Ratio')
    
    df_final = pd.merge(df_raw, median_stats, on='Jenis_Alat', how='left')
    
    # 3. Status
    def get_status(row):
        if row['Total_HM_Work'] == 0 or row['Total_Liter'] == 0:
            return "DATA TIDAK LENGKAP"
        
        bench = row['Group_Benchmark_Ratio']
        if pd.isna(bench): return "SINGLE UNIT"
        
        if row['Fuel_Ratio'] <= bench: return "Efisien"
        else: return "Boros"

    df_final['Performance_Status'] = df_final.apply(get_status, axis=1)
    
    # Formatting
    df_final['Fuel_Ratio'] = df_final['Fuel_Ratio'].round(2)
    df_final['Group_Benchmark_Ratio'] = df_final['Group_Benchmark_Ratio'].round(2)
    df_final.sort_values(['Jenis_Alat', 'Fuel_Ratio'], inplace=True)
    
    # 4. Summary Table
    summary_list = []
    for jenis, group in df_final.groupby('Jenis_Alat'):
        valid_grp = group[group['Performance_Status'].isin(["Efisien", "Boros"])]
        populasi = len(valid_grp)
        
        if populasi > 0:
            best = valid_grp.iloc[0]
            worst = valid_grp.iloc[-1]
            summary_list.append({
                'Jenis_Alat': jenis,
                'Populasi_Valid': populasi,
                'Benchmark_Median': best['Group_Benchmark_Ratio'],
                'Best_Unit': best['Unit_Name'],
                'Best_Ratio': best['Fuel_Ratio'],
                'Worst_Unit': worst['Unit_Name'] if populasi > 1 else "-",
                'Worst_Ratio': worst['Fuel_Ratio'] if populasi > 1 else "-"
            })
            
    df_summary = pd.DataFrame(summary_list)
    
    # 5. Save
    output_file = 'Analisa_Benchmark_Per_Alat_Berat.xlsx'
    try:
        with pd.ExcelWriter(output_file) as writer:
            df_summary.to_excel(writer, sheet_name='Summary_Per_Jenis', index=False)
            
            cols = ['Jenis_Alat', 'Unit_Name', 'Fuel_Ratio', 'Group_Benchmark_Ratio', 'Performance_Status', 'Total_Liter', 'Total_HM_Work']
            df_final[cols].to_excel(writer, sheet_name='Rapor_Per_Unit', index=False)
            
        print(f"\n💾  SUKSES! Laporan tersimpan: '{output_file}'")
        print("\n--- PREVIEW SUMMARY ---")
        print(df_summary.head(10).to_string(index=False))
        
    except Exception as e:
        print(f"⚠️ Gagal save Excel: {e}")
else:
    print("❌ Tidak ada data.")

🚀 [3/3] MEMBUAT LAPORAN EXCEL...



💾  SUKSES! Laporan tersimpan: 'Analisa_Benchmark_Per_Alat_Berat.xlsx'

--- PREVIEW SUMMARY ---
   Jenis_Alat  Populasi_Valid  Benchmark_Median                   Best_Unit  Best_Ratio                 Worst_Unit Worst_Ratio
     BULDOZER               1              0.52                      LANDAK        0.52                          -           -
        CRANE               5              6.03                        RG01        5.79 CRANE LB 80T LUCKY DOLPHIN        6.67
     FORKLIFT              62              3.00                       PALEM        0.67                    MAGNETO       11.77
REACH STACKER              32             14.77          KALAMR5 KINGKONG 1        9.12                  KALMAR 20       20.94
  SIDE LOADER               5              7.69          SIDE LOADER BOSS 3        0.64         SIDE LOUDER KALMAR        8.43
   TOP LOADER               5             13.11 TOP LOADER MITS CENDRAWASIH        0.21  TOP LOADER MITS 42T BADAK       22.32
      TRAILER  

In [14]:
# ==============================================================================
# DIAGNOSA: AUDIT UNIT YANG HILANG DARI ANALISA
# ==============================================================================
print("🚀 MENJALANKAN DIAGNOSA UNIT HILANG...")

# 1. Ambil List Semua Unit Target (Dari Master Mapping)
if 'unit_type_map' not in locals():
    # Reload jika perlu
    mapping_file = 'Master_Mapping_Jenis_Alat.xlsx'
    df_map = pd.read_excel(mapping_file)
    target_units = set(df_map[df_map['Jenis_Alat'] != 'UNKNOWN']['Unit_Name_BBM'].unique())
else:
    target_units = set(unit_type_map.keys())

print(f"1️⃣  Total Unit Target (Punya Jenis Alat): {len(target_units)} unit")

# 2. Ambil List Unit yang Lolos Analisa (Dari df_final Block 3)
if 'df_final' in locals():
    passed_units = set(df_final['Unit_Name'].unique())
else:
    passed_units = set()

print(f"2️⃣  Total Unit Lolos Analisa: {len(passed_units)} unit")

# 3. Cari Selisih (Unit yang Hilang)
missing_units = target_units - passed_units
print(f"3️⃣  Total Unit Terbuang/Hilang: {len(missing_units)} unit")

# 4. Investigasi Kenapa Hilang (Cek Data Mentah di unit_perf_data)
# Kita convert list unit_perf_data ke DataFrame dulu untuk cek
if 'unit_perf_data' in locals() and unit_perf_data:
    df_raw_check = pd.DataFrame(unit_perf_data)
    
    # Agregasi per unit (karena di raw data bisa banyak baris per unit)
    df_check_agg = df_raw_check.groupby('Unit_Name').agg({
        'Total_Liter': 'sum',
        'Total_HM_Work': 'sum'
    }).reset_index()
    
    report_missing = []
    
    for unit in missing_units:
        # Cek apakah unit ini ada di data mentah (BBM)?
        record = df_check_agg[df_check_agg['Unit_Name'] == unit]
        
        if record.empty:
            reason = "TIDAK ADA TRANSAKSI (Kosong di BBM)"
            detil = "-"
        else:
            liters = record.iloc[0]['Total_Liter']
            hm = record.iloc[0]['Total_HM_Work']
            
            if hm <= 0 and liters <= 0:
                reason = "DATA NULL (HM 0 & BBM 0)"
                detil = f"HM={hm}, BBM={liters}"
            elif hm <= 0:
                reason = "HM ERROR (HM tidak bergerak/0)"
                detil = f"HM={hm}, BBM={liters}"
            elif liters <= 0:
                reason = "TIDAK ADA BBM (BBM 0)"
                detil = f"HM={hm}, BBM={liters}"
            else:
                reason = "UNKNOWN ERROR"
                detil = f"HM={hm}, BBM={liters}"
        
        # Ambil Jenis Alat
        jenis = unit_type_map.get(unit, "UNKNOWN")
        
        report_missing.append({
            'Unit_Name': unit,
            'Jenis_Alat': jenis,
            'Alasan_Dibuang': reason,
            'Detail_Angka': detil
        })
        
    # 5. Export Laporan Missing
    if report_missing:
        df_missing_report = pd.DataFrame(report_missing)
        
        # Sortir biar rapi
        df_missing_report.sort_values(['Alasan_Dibuang', 'Unit_Name'], inplace=True)
        
        output_file_missing = 'Unit_Exceptional_Analisa_Per_Alat_Berat.xlsx'
        df_missing_report.to_excel(output_file_missing, index=False)
        
        print(f"\n⚠️  DITEMUKAN {len(df_missing_report)} UNIT BERMASALAH.")
        print(f"    File Laporan: '{output_file_missing}'")
        print("\n--- CONTOH 10 UNIT YANG DIBUANG ---")
        print(df_missing_report[['Unit_Name', 'Alasan_Dibuang', 'Detail_Angka']].head(10).to_string(index=False))
    else:
        print("✅ Aneh, tidak ada report missing padahal ada selisih angka.")

else:
    print("❌ Data mentah 'unit_perf_data' tidak ditemukan. Jalankan Block 2 dulu.")

🚀 MENJALANKAN DIAGNOSA UNIT HILANG...
1️⃣  Total Unit Target (Punya Jenis Alat): 242 unit
2️⃣  Total Unit Lolos Analisa: 191 unit
3️⃣  Total Unit Terbuang/Hilang: 51 unit

⚠️  DITEMUKAN 51 UNIT BERMASALAH.
    File Laporan: 'Unit_Exceptional_Analisa_Per_Alat_Berat.xlsx'

--- CONTOH 10 UNIT YANG DIBUANG ---
               Unit_Name                 Alasan_Dibuang       Detail_Angka
               L 9743 UR HM ERROR (HM tidak bergerak/0)  HM=0.0, BBM=170.0
                    SOKA HM ERROR (HM tidak bergerak/0)  HM=0.0, BBM=55.58
B 9137 MJ (EX B 9494 JA)          TIDAK ADA BBM (BBM 0) HM=4616.0, BBM=0.0
               B 9157 ZJ          TIDAK ADA BBM (BBM 0) HM=4375.0, BBM=0.0
               B 9220 BL          TIDAK ADA BBM (BBM 0) HM=4134.0, BBM=0.0
               B 9224 BL          TIDAK ADA BBM (BBM 0) HM=3782.0, BBM=0.0
               B 9493 JA          TIDAK ADA BBM (BBM 0) HM=4311.0, BBM=0.0
               B 9566 ZB          TIDAK ADA BBM (BBM 0) HM=4399.0, BBM=0.0
               B 